In [2]:
from pytorchcv.model_provider import get_model as ptcv_get_model

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms
from di import DeepInversionClass

In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='/dev/shm', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='/dev/shm', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [7]:
def test(model, loader):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    test_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    loss = test_loss / (batch_idx + 1)
    acc = 100.* correct / total
    return loss, acc

def update_bn(model, loader):
    model.train()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)

def test_on_gen(model, inputs, targets):
    model.eval()
    with torch.no_grad():
        inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total = targets.size(0)
        correct = predicted.eq(targets).sum().item()
    return 100.* correct / total

In [4]:
teacher = ptcv_get_model('resnet20_cifar10', pretrained=True).cuda()
test(teacher, testloader)

(0.19714343681931495, 94.03)

In [5]:
student = ptcv_get_model('nin_cifar10', pretrained=True).cuda()
test(student, testloader)

(0.3114712432026863, 92.57)

In [6]:
inv_batchsize = 100

In [7]:
di= DeepInversionClass(bs=inv_batchsize, net_teacher=teacher, path='./test')
di.bn_reg_scale = 1.0
inv_image, inv_target = di.get_images(net_student=None, 
                                      reset_inputs=True, reset_targets=True, reset_opt=True, iterations_per_layer=4000, quant_input=False)

print(test_on_gen(teacher, inv_image, inv_target))
print(test_on_gen(student, inv_image, inv_target))

-------------------- 2000 ------------------
R_feature: 7.598e+00, R_cross: 1.431e-02, R_total: 7.663e+00
-------------------- 4000 ------------------
R_feature: 6.014e+00, R_cross: 1.546e-02, R_total: 6.081e+00
100.0
89.0


In [8]:
di= DeepInversionClass(bs=inv_batchsize, net_teacher=teacher, path='./test')
di.bn_reg_scale = 0.0
inv_image, inv_target = di.get_images(net_student=None, 
                                      reset_inputs=True, reset_targets=True, reset_opt=True, iterations_per_layer=4000, quant_input=False)

print(test_on_gen(teacher, inv_image, inv_target))
print(test_on_gen(student, inv_image, inv_target))

-------------------- 2000 ------------------
R_feature: 9.293e+01, R_cross: 2.859e-05, R_total: 7.943e-03
-------------------- 4000 ------------------
R_feature: 9.187e+01, R_cross: 8.090e-05, R_total: 1.079e-02
100.0
14.0


In [8]:
random_model = ptcv_get_model('resnet20_cifar10', pretrained=False).cuda()
# for n, m in random_model.named_modules():
#     if isinstance(m, nn.BatchNorm2d):
#         print(n)
print(test(random_model, testloader))
for i in range(20):
    print(random_model.features.init_block.bn.running_mean)
    update_bn(random_model, trainloader)
    print(i, test(random_model, testloader))

# di= DeepInversionClass(bs=inv_batchsize, net_teacher=random_model, main_teacher=teacher, path='./test')
# di.bn_reg_scale = 1.0
# inv_image, inv_target = di.get_images(net_student=None, 
#                                       reset_inputs=True, reset_targets=True, reset_opt=True, iterations_per_layer=4000, quant_input=False)

# print(test_on_gen(teacher, inv_image, inv_target))
# print(test_on_gen(student, inv_image, inv_target))

(26.241160984039308, 11.76)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
0 (2.5009267210960386, 9.65)
tensor([-0.4132, -0.3816,  0.0317,  0.1204,  0.5954, -0.3230, -1.2533,  0.0660,
        -0.4014, -0.2019,  0.0796, -0.6648, -0.1595, -0.1209,  0.0655,  0.2131],
       device='cuda:0')
1 (2.4947407603263856, 9.65)
tensor([-0.3717, -0.3412,  0.0327,  0.1074,  0.5356, -0.2870, -1.1218,  0.0641,
        -0.3584, -0.1816,  0.0687, -0.6014, -0.1338, -0.1084,  0.0554,  0.1898],
       device='cuda:0')
2 (2.5012975192070006, 9.69)
tensor([-0.4237, -0.3897,  0.0227,  0.1210,  0.6046, -0.3345, -1.2824,  0.0610,
        -0.4102, -0.2058,  0.0854, -0.6796, -0.1661, -0.1238,  0.0684,  0.2159],
       device='cuda:0')
3 (2.497236740589142, 9.71)
tensor([-0.4044, -0.3771,  0.0337,  0.1154,  0.5808, -0.3055, -1.2206,  0.0651,
        -0.3916, -0.1964,  0.0775, -0.6560, -0.1433, -0.1190,  0.0554,  0.2022],
       device='cuda:0')
4 (2.502112386226654

In [10]:
random_model = ptcv_get_model('seresnet56_cifar10', pretrained=True).cuda()
print(test(random_model, testloader))
# for i in range(20):
#     update_bn(random_model, trainloader)
#     print(i, test(random_model, testloader))

di= DeepInversionClass(bs=inv_batchsize, net_teacher=random_model, main_teacher=teacher, path='./test')
di.bn_reg_scale = 1.0
inv_image, inv_target = di.get_images(net_student=None, 
                                      reset_inputs=True, reset_targets=True, reset_opt=True, iterations_per_layer=4000, quant_input=False)

print(test_on_gen(teacher, inv_image, inv_target))
print(test_on_gen(student, inv_image, inv_target))

(0.1668540365062654, 95.87)
-------------------- 2000 ------------------
R_feature: 1.376e+01, R_cross: 1.673e-02, R_total: 1.384e+01
-------------------- 4000 ------------------
R_feature: 1.067e+01, R_cross: 1.725e-02, R_total: 1.074e+01
100.0
73.0
